<h1>Testing Twitter API and getting tweet data</h1>

Things needed in the dataset
- Tweet count for each "keyword"
- Tweet count for each "keyword's" sentiment analysis (positive, negative, neutral)
- Possible hover data for interactivity: Country of Origin top 5 ranked??

In [21]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob
import json

In [22]:
allCurrencies = [
    {
        "currency": "Bitcoin BTC",
        "keywords": ["Bitcoin", "bitcoin", "BTC", "btc"]
    },
    {
        "currency": "Ethereum ETH",
        "keywords": ["Ethereum", "ethereum", "ETH", "eth"]
    },
    {
        "currency": "Ripple XRP",
        "keywords": ["Ripple", "ripple", "XRP", "xrp"]
    },
    {
        "currency": "Litecoin LTC",
        "keywords": ["LiteCoin", "litecoin", "LTC", "ltc"]
    },
    {
        "currency": "Bitcoin Cash BCH",
        "keywords": ["BitcoinCash", "bitcoincash", "BCH", "bch"]
    }
]

In [23]:
class TwitterClient(object): 
    ''' 
    Generic Twitter Class for sentiment analysis. 
    '''
    def __init__(self): 
        ''' 
        Class constructor or initialization method. 
        '''
        # keys and tokens from the Twitter Dev Console 
        consumer_key = 'flGjLnMA10iavQTzNQrDPpwti'
        consumer_secret = '0OVASwhRSym2arTiCaWa7AM8K5D9vpfEcoEIsWuDWpXSAO7DCx'
        access_token = '489133000-YmOfIQ5V9GjKVDkMv8Z1tqQLYjvRLIJlDUJvv0Od'
        access_token_secret = '5VFPev3qfK19NHZai1UTM871SWWVr1uIWTkFAIblieu3B'

        # attempt authentication 
        try: 
            # create OAuthHandler object 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed") 

    def clean_tweet(self, tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 

    def get_tweet_sentiment(self, tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(self.clean_tweet(tweet)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'

    def get_tweets(self, query, count = 10): 
        ''' 
        Main function to fetch tweets and parse them. 
        '''
        # empty list to store parsed tweets 
        tweets = [] 

        try: 
            # call twitter api to fetch tweets 
            fetched_tweets = self.api.search(q = query, count = count) 

            # parsing tweets one by one 
            for tweet in fetched_tweets: 
                # empty dictionary to store required params of a tweet 
                parsed_tweet = {} 

                # saving text of tweet 
                parsed_tweet['text'] = tweet.text 
                # saving sentiment of tweet 
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 
                # saving number of retweets on the tweet
                parsed_tweet['retweets'] = tweet.retweet_count
                # saving number of likes on the tweet
                parsed_tweet['likes'] = tweet.favorite_count

                # appending parsed tweet to tweets list 
                if tweet.retweet_count > 0: 
                    # if tweet has retweets, ensure that it is appended only once 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 

            # return parsed tweets 
            return tweets

        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e)) 

In [24]:
def getData(data, total, tweets):    
    obj = {
        'name': data['currency']
    }
    
    # positive data
    # tweets
    posT = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
    # total retweets 
    rtweetsP = [tweet['retweets'] for tweet in posT] 
    posRet = sum(rtweetsP)
    # total favorites 
    ftweetsP = [tweet['likes'] for tweet in posT] 
    posFav = sum(ftweetsP)
    # positive fraction
    posFrac = len(posT)/total
    posObj = {
        'name': 'Positive',
        'value': posFrac*100
    }
    
    # negative data
    # tweets
    negT = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    # total retweets 
    rtweetsN = [tweet['retweets'] for tweet in negT]
    negRet = sum(rtweetsN)
    # total favorites 
    ftweetsN = [tweet['likes'] for tweet in negT] 
    negFav = sum(ftweetsN)
    # negative fraction
    negFrac = len(negT)/total
    negObj = {
        'name': 'Negative',
        'value': negFrac*100
    }
    
    # neutral data
    # tweets
    neutT = [tweet for tweet in tweets if (tweet['sentiment'] != 'negative' and tweet['sentiment'] != 'positive')]
    # total retweets 
    rtweetsNeu = [tweet['retweets'] for tweet in neutT] 
    neutRet = sum(rtweetsNeu)
    # total favorites 
    ftweetsNeu = [tweet['likes'] for tweet in neutT] 
    neutFav = sum(ftweetsNeu)
    # positive fraction
    neutFrac = len(neutT)/total
    neutObj = {
        'name': 'Neutral',
        'value': neutFrac*100
    }
    
    obj['children'] = [posObj, negObj, neutObj]
    obj['value'] = (posFrac + negFrac + neutFrac)*100
    
    return obj

In [25]:
def main():     
    allTweets = []
    for data in allCurrencies:
        api = TwitterClient()
        tweets = api.get_tweets(query = data['keywords'], count = 1000)
        allTweets.append(tweets)
        
    totalTweets = 0
    for currency in allTweets:
        totalTweets += len(currency)
        
    allDicts = []
    for i in range(0, len(allCurrencies)):
        allDicts.append(getData(allCurrencies[i], totalTweets, allTweets[i]))
    
    final = {
        'name': 'Cryptocurrencies',
        'children': allDicts
    }
    with open('cryptoTweetData.json', 'w') as fp:
        json.dump(final, fp, indent=4, sort_keys=True)
        

In [26]:
if __name__ == "__main__": 
    # calling main function 
    main() 